In [1]:
# !pip install bs4

In [2]:
# !pip install requests

In [2]:
# import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
# get movie titles to scrap
get_request = requests.get("https://imsdb.com/genre/Family")
bs = BeautifulSoup(get_request.text, "html.parser")
interest_section = bs.findAll("td", {"valign" : "top"})[2]

movie_titles = interest_section.findAll("a")
movie_titles = [title.text for title in movie_titles if title.attrs['href'].startswith("/Movie Scripts/") and title.attrs['href'].endswith(".html")]

print(len(movie_titles))
movie_titles[:10]

54


['Addams Family, The',
 'Aladdin',
 'Anastasia',
 'Back to the Future',
 'Back to the Future II & III',
 'Beauty and the Beast',
 'Big',
 'Chronicles of Narnia: The Lion, the Witch and the Wardrobe',
 'Coraline',
 'Duck Soup']

In [107]:
# reducing the titles just for test
movie_titles = movie_titles[-5:] + ['Up']
movie_titles

['Wall-E',
 'Who Framed Roger Rabbit?',
 'Wizard of Oz, The',
 'Kung Fu Panda',
 'Wall-E',
 'Up']

In [108]:
# scrap movie scripts
for movie_title in movie_titles:
    
    # get the html
    link_movie_title = movie_title.replace(' ', '-')
    get_request = requests.get("https://imsdb.com/scripts/" + link_movie_title + ".html")
    bs = BeautifulSoup(get_request.text, "html.parser")

    # some very few scripts have different HTML format, let's ignore them for now
    if len(bs.findAll("pre")) == 0:
        continue
    
    # scrap the entire script
    script_ = bs.findAll("pre")[0].text
    script_ = script_.split("\r")
    
    # display movie title and first 10 lines of the script
    print("MOVIE:", movie_title)
    display(script_[:10])
    print("=" * 30)

MOVIE: Wall-E


['',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n                                    WALL-E',
 '\n',
 '\n',
 '\n                                  Written by',
 '\n']

MOVIE: Wizard of Oz, The


['FADE IN -- Title:',
 '\n',
 '\nFor nearly forty years this story has given faithful service',
 '\nto the Young in Heart; and Time has been powerless to put its',
 '\nkindly philosophy out of fashion.',
 '\n',
 '\nTo those of you who have been faithful to it in return',
 '\n',
 '\n...and to the Young in Heart --- we dedicate this picture.',
 '\n']

MOVIE: Kung Fu Panda


['',
 '\n',
 '\n',
 '\n',
 '\n                             KUNG FU PANDA',
 '\n          ',
 '\n          ',
 '\n',
 '\n                              Written By',
 '\n          ']

MOVIE: Wall-E


['',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n                                    WALL-E',
 '\n',
 '\n',
 '\n                                  Written by',
 '\n']

MOVIE: Up


[' ',
 '\n          ',
 '\n',
 '\n                                       UP',
 '\n',
 '\n          ',
 '\n                                   Written by',
 '\n          ',
 '\n                   Pete Docter, Bob Peterson & Thomas McCarthy',
 '\n          ']

In [113]:
# get characters from movie script
script = script_[:]
character_start = '\n                              '

characters = list(set([line.strip() for line in script if line.startswith(character_start) and line[len(character_start)] != ' ']))
characters = list(set([ch.split('(')[0].strip() for ch in characters]))
if '' in characters: characters.remove('')

characters

['CONSTRUCTION WORKER STEVE',
 'CARL',
 'YOUNG CARL',
 'YOUNG ELLIE',
 "GIRL'S VOICE",
 'DOGS',
 'NEWSREEL ANNOUNCER',
 'VOICE',
 'CAMP MASTER STRAUCH',
 'MUNTZ',
 'ALPHA',
 'OFFICER EDITH',
 'DOG WALLA',
 'CROWD',
 'OMEGA',
 'BETA',
 'CONSTRUCTION FOREMAN TOM',
 'RUSSELL',
 'CARL/RUSSELL',
 'GAMMA',
 'MISC DOG',
 'DUG',
 'NURSE GEORGE']

In [8]:
# get title from movie script
# === not required for now ===

# script = script_[:]
# script = list(map(str.strip, script))
# script = [line for line in script if len(line) != 0]

# title = script[0]
# title

In [114]:
# get lines from movie script
script = script_[:]

lines = []
characters_tuple = tuple(characters)
saving_line, character_line = False, ""
for i, line in enumerate(script):
    
    if line.strip().startswith(characters_tuple):
        # print(i, line.strip())
        character = [ch for ch in characters_tuple if line.strip().startswith(ch)]
        character = max(character, key=len)
        saving_line, character_line = True, ""
        continue
        
    if saving_line and line.strip() != "":
        character_line += line.strip() + " "
        continue
        
    if saving_line and line.strip() == "":
        saving_line = False
        lines.append((movie_title, character, character_line.rstrip()))
        
lines[:10]

[('Up',
  'NEWSREEL ANNOUNCER',
  '"Movietown News" presents... Spotlight on Adventure!'),
 ('Up',
  'NEWSREEL ANNOUNCER',
  'What you are now witnessing is footage never before seen by civilized humanity: a lost world in South America! Lurking in the shadow of majestic Paradise Falls, it sports plants and animals undiscovered by science. Who would dare set foot on this inhospitable summit?'),
 ('Up', 'NEWSREEL ANNOUNCER', 'Why, our subject today: Charles Muntz!'),
 ('Up',
  'NEWSREEL ANNOUNCER',
  'The beloved explorer lands his dirigible, the "Spirit of Adventure," in New Hampshire this week, completing a year long expedition to the lost world!'),
 ('Up',
  'NEWSREEL ANNOUNCER',
  'This lighter-than-air craft was designed by Muntz himself, and is longer than 22 Prohibition paddy- wagons placed end to end.'),
 ('Up', 'NEWSREEL ANNOUNCER', 'And here comes the adventurer now!'),
 ('Up',
  'NEWSREEL ANNOUNCER',
  "Never apart from his faithful dogs, Muntz conceived the craft for canine c

In [115]:
# creating df from character lines of movie script
df = pd.DataFrame(lines, columns =['movie_title', 'character', 'line'])
df

,movie_title,character,line
0,Up,NEWSREEL ANNOUNCER,"""Movietown News"" presents... Spotlight on Adve..."
1,Up,NEWSREEL ANNOUNCER,What you are now witnessing is footage never b...
2,Up,NEWSREEL ANNOUNCER,"Why, our subject today: Charles Muntz!"
3,Up,NEWSREEL ANNOUNCER,"The beloved explorer lands his dirigible, the ..."
4,Up,NEWSREEL ANNOUNCER,This lighter-than-air craft was designed by Mu...
...,...,...,...
710,Up,CARL,"It's red, isn't it?"
711,Up,RUSSELL,"Mr. Fredricksen, you're cheating."
712,Up,CARL,"No, I'm not. Red one."
713,Up,RUSSELL,That's a fire hydrant.


In [116]:
df['character'].value_counts()

CARL                         243
RUSSELL                      211
MUNTZ                         60
DUG                           50
YOUNG ELLIE                   31
ALPHA                         23
NEWSREEL ANNOUNCER            21
GAMMA                         15
BETA                          11
VOICE                          8
CONSTRUCTION FOREMAN TOM       7
DOG WALLA                      6
CAMP MASTER STRAUCH            5
CONSTRUCTION WORKER STEVE      4
GIRL'S VOICE                   4
YOUNG CARL                     4
NURSE GEORGE                   3
OFFICER EDITH                  2
DOGS                           2
OMEGA                          2
CROWD                          1
MISC DOG                       1
CARL/RUSSELL                   1
Name: character, dtype: int64

In [117]:
# filter characters that have less than "n" lines
min_lines = 30
df = df.groupby('character').filter(lambda x: len(x) >= min_lines)
df.reset_index(drop=True, inplace=True)
df

,movie_title,character,line
0,Up,MUNTZ,"""Adventure is out there!"""
1,Up,MUNTZ,"Gentlemen, I give you: the Monster of Paradise..."
2,Up,MUNTZ,I promise to capture the beast... alive!
3,Up,MUNTZ,And I will not come back until I do! The crowd...
4,Up,YOUNG ELLIE,"It's a beautiful day, winds out of the east at..."
...,...,...,...
590,Up,CARL,"It's red, isn't it?"
591,Up,RUSSELL,"Mr. Fredricksen, you're cheating."
592,Up,CARL,"No, I'm not. Red one."
593,Up,RUSSELL,That's a fire hydrant.


In [119]:
df['character'].value_counts()

CARL           243
RUSSELL        211
MUNTZ           60
DUG             50
YOUNG ELLIE     31
Name: character, dtype: int64

In [3]:
df1 = pd.read_csv("Kung_Fu_Panda.csv")
df2 = pd.read_csv("Wall_E.csv")
df3 = pd.read_csv("Up.csv")

df = pd.concat([df1, df2, df3], ignore_index=True)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')] # temp way to drop unnamed column
df

,movie_title,character,line
0,Kung Fu Panda,TIGRESS,You'll be late for work!
1,Kung Fu Panda,PO,Whu?
2,Kung Fu Panda,PO'S DAD,Po! Get up!
3,Kung Fu Panda,PO'S DAD,Po! What are you doing up there?
4,Kung Fu Panda,PO,"Uh, nothing!"
...,...,...,...
1439,Up,CARL,"It's red, isn't it?"
1440,Up,RUSSELL,"Mr. Fredricksen, you're cheating."
1441,Up,CARL,"No, I'm not. Red one."
1442,Up,RUSSELL,That's a fire hydrant.


In [131]:
df[df["line"] == ""]

,movie_title,character,line


In [5]:
# 1st approach using spacy for document similarty
import spacy
import spicy

nlp = spacy.load("en_core_web_lg")

D:\Anaconda\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [24]:
user_input = "I can survive for months"
user_doc = nlp(user_input)


sims = []

for i, line in enumerate(df['line'].to_list()):
    if not isinstance(line, str) or line == "": continue
    
    # can also add docs as a column beforehand to reduce loop processing time
    doc = nlp(line)
    sim = user_doc.similarity(doc)
    
    sims.append((i, sim))

sims[:10]

[(0, 0.7183285756031559),
 (1, 0.2585359866767033),
 (2, 0.2259159488271546),
 (3, 0.5992965980249952),
 (4, 0.4797644645302548),
 (5, 0.2839090470040722),
 (6, 0.5482183680655116),
 (7, 0.24324737931497348),
 (8, 0.45647720359456756),
 (9, 0.6516046821805714)]

In [25]:
sims = sorted(sims, key= lambda x:x[1], reverse=True)
sims[:10]

[(312, 0.8461008223028246),
 (462, 0.8424039281767293),
 (363, 0.8371495200434744),
 (21, 0.8360245159806813),
 (1390, 0.8311597498990037),
 (80, 0.8289876457724381),
 (319, 0.8261917116094339),
 (173, 0.8195854761776106),
 (460, 0.8178320916366182),
 (392, 0.8140147891652875)]

In [26]:
df.dropna(inplace=True)

In [27]:
df.isna().sum()

movie_title    0
character      0
line           0
dtype: int64

In [28]:
# show top 5 lines from characters similar to the user_input text
for i, sim in sims[:10]:
    print(sim)
    print("Phrase:", list(df.iloc[[i]]['line']))
    display(df.iloc[[i]])

0.8461008223028246
Phrase: ['And I can control--']


,movie_title,character,line
312,Kung Fu Panda,SHIFU,And I can control--


0.8424039281767293
Phrase: ['Then I will finally have paid for my mistake.']


,movie_title,character,line
462,Kung Fu Panda,SHIFU,Then I will finally have paid for my mistake.


0.8371495200434744
Phrase: ['But I can change you! I can turn you into the Dragon Warrior! And I will!']


...

,movie_title,character,line
173,Kung Fu Panda,SHIFU,"Now listen closely, panda. Oogway may have pic..."


0.8178320916366182
Phrase: ['I can hold him off long enough for everyone to escape.']


,movie_title,character,line
460,Kung Fu Panda,SHIFU,I can hold him off long enough for everyone to...


0.8140147891652875
Phrase: ['But perhaps that is my fault. I cannot train you the way I have trained the Five. I now see that the way to get through to you is with this!']


,movie_title,character,line
392,Kung Fu Panda,SHIFU,But perhaps that is my fault. I cannot train y...


In [29]:
# 2nd approach, using Tfidf for pairwise similarity
from sklearn.feature_extraction.text import TfidfVectorizer

docs = df['line'].to_list()
vect = TfidfVectorizer(min_df=1, stop_words="english")
tfidf = vect.fit_transform(docs)

In [30]:
# just to visualize the tfidf of our dataframe
col_names = vect.get_feature_names_out()
arr = tfidf.toarray()
df_test = pd.DataFrame(arr, columns = col_names)
df_test

,100,12,2110,255,30,54,642,700,700th,72,...,yep,yes,yesterday,yikes,york,young,yow,zeng,zero,zoology
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
tfidf.shape

(1411, 1954)

In [32]:
user_vect = TfidfVectorizer(min_df=1, stop_words="english")
user_tfidf = user_vect.fit_transform([user_input])

In [33]:
# just to visualize the tfidf of the user input
col_names = user_vect.get_feature_names_out()
arr = user_tfidf.toarray()
df_test_user = pd.DataFrame(arr, columns = col_names)
df_test_user

,months,survive
0,0.707107,0.707107


In [34]:
user_tfidf.shape

(1, 2)

In [35]:
df_test_user

,months,survive
0,0.707107,0.707107


In [36]:
df_concat = pd.concat([df_test, df_test_user], ignore_index=True)
df_concat = df_concat.fillna(0.0)
df_concat

,100,12,2110,255,30,54,642,700,700th,72,...,yep,yes,yesterday,yikes,york,young,yow,zeng,zero,zoology
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
df_test_user = df_concat.iloc[[-1]]
df_test_user

,100,12,2110,255,30,54,642,700,700th,72,...,yep,yes,yesterday,yikes,york,young,yow,zeng,zero,zoology
1411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
user_tfidf = spicy.sparse.csr_matrix(df_test_user.values)

In [39]:
display(tfidf.shape)
display(user_tfidf.shape)

(1411, 1954)

(1, 1954)

In [40]:
pairwise_similarity = tfidf * user_tfidf.T
pairwise_similarity

<1411x1 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [41]:
sims = [ (i, matrix.item(0,0)) for i, matrix in enumerate(list(pairwise_similarity.todense()))]

# sort similarities
sims = sorted(sims, key = lambda x : x[1], reverse=True)

# filter away zero-value similarities
sims = list(filter(lambda x : x[1] != 0.0, sims))

# show top 5 lines from characters similar to the user_input text
print("User Input:", [user_input])
for i, sim in sims[:10]:
    print(sim)
    print("Phrase:", list(df.iloc[[i]]['line']))
    display(df.iloc[[i]])

User Input: ['I can survive for months']
0.452088827516804
Phrase: ['It is said that the Dragon Warrior can survive for months at a time on nothing but the dew of a single gingko leaf and the energy of the universe.']


,movie_title,character,line
332,Kung Fu Panda,TIGRESS,It is said that the Dragon Warrior can survive...


0.35019579924980165
Phrase: ["I DON'T WANT TO SURVIVE! I WANT TO LIVE!"]


,movie_title,character,line
759,Wall-E,CAPTAIN,I DON'T WANT TO SURVIVE! I WANT TO LIVE!
